# Value types
Nix 2.0 contains a command named nix repl which is a simple command line tool for playing with the Nix language. In fact, Nix is a pure, lazy, functional language, not only a set of tools to manage derivations. The nix repl syntax is slightly different to Nix syntax when it comes to assigning variables, but it shouldn't be confusing so long as you bear it in mind. I prefer to start with nix repl before cluttering your mind with more complex expressions.

Launch nix repl. First of all, Nix supports basic arithmetic operations: +, -, * and /. (To exit nix repl, use the command :q. Help is available through the :? command.)

In [1]:
1+3
7-4
3*2

1+3
4

7-4
3

3*2
6



Attempting to perform division in Nix can lead to some surprises.

In [2]:
6.3

6.3
6.3



What happened? Recall that Nix is not a general purpose language, it's a domain-specific language for writing packages. Integer division isn't actually that useful when writing package expressions. Nix parsed 6/3 as a relative path to the current directory. To get Nix to perform division instead, leave a space after the /. Alternatively, you can use builtins.div.

In [3]:
6/ 3

6/ 3
2



In [4]:
builtins.div 6 3

builtins.div 6 3
2



Other operators are ||, && and ! for booleans, and relational operators such as !=, ==, <, >, <=, >=. In Nix, <, >, <= and >= are not much used. There are also other operators we will see in the course of this series.

Nix has integer, floating point, string, path, boolean and null simple types. Then there are also lists, sets and functions. These types are enough to build an operating system.

Nix is strongly typed, but it's not statically typed. That is, you cannot mix strings and integers, you must first do the conversion.

As demonstrated above, expressions will be parsed as paths as long as there's a slash not followed by a space. Therefore to specify the current directory, use ./. In addition, Nix also parses urls specially.

Not all urls or paths can be parsed this way. If a syntax error occurs, it's still possible to fallback to plain strings. Literal urls and paths are convenient for additional safety.

# Identifier
There's not much to say here, except that dash (-) is allowed in identifiers. That's convenient since many packages use dash in their names. In fact:

In [5]:
a-b

a-b
error: undefined variable 'a-b' at (string):1:1



In [6]:
a - b

a - b
error: undefined variable 'a' at (string):1:1



As you can see, a-b is parsed as identifier, not as a subtraction.

# Strings
It's important to understand the syntax for strings. When learning to read Nix expressions, you may find dollars ($) ambiguous, but they are very important . Strings are enclosed by double quotes ("), or two single quotes ('').

In [7]:
"foo"

"foo"
"foo"



In [8]:
''foo''

''foo''
"foo"



In other languages like Python you can also use single quotes for strings (e.g. 'foo'), but not in Nix.

It's possible to interpolate whole Nix expressions inside strings with the ${...} syntax and only that syntax, not $foo or {$foo} or anything else.

In [9]:
foo = "strval"

foo = "strval"



In [10]:
"$foo"

"$foo"
"$foo"



In [11]:
"${foo}"

"${foo}"
"strval"



In [12]:
"${2+3}"

"${2+3}"
error: cannot coerce an integer to a string, at (string):1:2



Note: ignore the foo = "strval" assignment, special syntax in nix repl.

As said previously, you cannot mix integers and strings. You need to explicitly include conversions. We'll see this later: function calls are another story.

Using the syntax with two single quotes is useful for writing double quotes inside strings without needing to escape them:

In [13]:
''test " test''

''test " test''
"test \" test"



In [14]:
''${foo}''

''${foo}''
"strval"



Escaping ${...} within double quoted strings is done with the backslash. Within two single quotes, it's done with '':

In [15]:
"\${foo}"

"\${foo}"
"${foo}"



In [16]:
''test ''${foo} test''

''test ''${foo} test''
"test ${foo} test"



# Lists
Lists are a sequence of expressions delimited by space (not comma):

In [17]:
[ 2 "foo" true (2+3) ]

[ 2 "foo" true (2+3) ]
[ 2 "foo" true 5 ]



Lists, like everything else in Nix, are immutable. Adding or removing elements from a list is possible, but will return a new list.

# Attribute sets
Attribute sets are an association between string keys and a Nix values. Keys can only be strings. When writing attribute sets you can also use unquoted identifiers as keys.

In [18]:
s = { foo = "bar"; a-b = "baz"; "123" = "num"; }
s

s = { foo = "bar"; a-b = "baz"; "123" = "num"; }

s
{ "123" = "num"; a-b = "baz"; foo = "bar"; }



For those reading Nix expressions from nixpkgs: do not confuse attribute sets with argument sets used in functions.

To access elements in the attribute set:

In [19]:
s.a-b
s."123"

s.a-b
"baz"

s."123"
"num"



Yes, you can use strings to address keys which aren't valid identifiers.

Inside an attribute set you cannot normally refer to elements of the same attribute set:

In [20]:
{ a = 3; b = a+4; }

{ a = 3; b = a+4; }
error: undefined variable 'a' at (string):1:14



To do so, use recursive attribute sets:

In [21]:
rec { a = 3; b = a+4; }

rec { a = 3; b = a+4; }
{ a = 3; b = 7; }



This is very convenient when defining packages, which tend to be recursive attribute sets.

# Let expressions
This kind of expression is used to define local variables for inner expressions.

In [22]:
let a = "foo"; in a

let a = "foo"; in a
"foo"



Let's write two let expressions, one inside the other:

In [23]:
let a = 3; b = 4; in a + b

let a = 3; b = 4; in a + b
7



With let you cannot assign twice to the same variable. However, you can shadow outer variables:m

In [24]:
let a = 3; a = 8; in a
let a = 3; in let a = 8; in a

let a = 3; a = 8; in a
error: attribute 'a' at (string):1:12 already defined at (string):1:5

let a = 3; in let a = 8; in a
8



You cannot refer to variables in a let expression outside of it:

In [25]:
let a = (let b = 3; in b); in b

let a = (let b = 3; in b); in b
error: undefined variable 'b' at (string):1:31



You can refer to variables in the let expression when assigning variables, like with recursive attribute sets:

In [26]:
let a = 4; b = a + 5; in b

let a = 4; b = a + 5; in b
9



So beware when you want to refer to a variable from the outer scope, but it's also defined in the current let expression. The same applies to recursive attribute sets.

# With expression
This kind of expression is something you rarely see in other languages. You can think of it like a more granular version of using from C++, or from module import * from Python. You decide per-expression when to include symbols into the scope.

In [27]:
longName = { a = 3; b = 4; }
longName.a + longName.b

longName = { a = 3; b = 4; }

longName.a + longName.b
7



In [28]:
 with longName; a + b

 with longName; a + b
7



That's it, it takes an attribute set and includes symbols from it in the scope of the inner expression. Of course, only valid identifiers from the keys of the set will be included. If a symbol exists in the outer scope and would also be introduced by the with, it will not be shadowed. You can however still refer to the attribute set:

In [29]:
let a = 10; in with longName; a + b

let a = 10; in with longName; a + b
14



In [30]:
let a = 10; in with longName; longName.a + b

let a = 10; in with longName; longName.a + b
7



# Laziness
Nix evaluates expression only when needed. This is a great feature when working with package

In [31]:
let a = builtins.div 4 0; b = 6; in b

let a = builtins.div 4 0; b = 6; in b
6



Since a is not needed, there's no error about division by zero, because the expression is not in need to be evaluated. That's why we can have all the packages defined on demand, yet have access to specific packages very quickly.